In [1]:
import tweepy
from tweepy import OAuthHandler
import csv
import cv2
import numpy as np
import sys
import random
import pandas as pd
import math
import glob
import os
import io
import re
import string
import operator
import itertools
from collections import Counter
from collections import defaultdict 
import json
import nltk
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI
from nltk.tokenize import word_tokenize
import urllib3
import svm
from svmutil import *
feature_list = []


In [2]:
def test_svm():
    labels = [0, 1, 1, 2]
    samples = [[0, 1, 0], [1, 1, 1], [1, 1, 0], [0, 0, 0]]

    #SVM params
    param = svm_parameter()
    param.C = 10
    param.kernel_type = SIGMOID
    #instantiate the problem
    problem = svm_problem(labels, samples)
    #train the model
    model = svm_train(problem, param)
    # saved model can be loaded as below
    #model = svm_load_model('model_file')

    #save the model
    svm_save_model('model_file', model)

    #test data
    test_data = [[0, 1, 1], [1, 0, 1],[1,1,1],[1,0,0],[0,0,0]]
    #predict the labels
    #predict takes 3 arguments: 1. test labels, 2. test data, 3. model
    p_labels, p_accs, p_vals = svm_predict([0,1,1,2,2], test_data, model)
    print p_labels
    print ("-------------------------------------------------------------------------------")
    

In [3]:
def map_features(tweet):
    words = set(tweet)
    features = {}
    for f in feature_list:
        features[f]= (f in words)
    return features

In [4]:
def remove_extra(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

In [5]:
def get_features(tweet):
    feature_vec = []
    words = tweet.split()
    for word in words:
        if ((not re.match(r'^https:',word)) and  (not re.match(r'^@',word))):
            word = remove_extra(word)
            word = word.strip('\'"?,.')
            val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", word)
            if (val is None) or (len(word)<=3):
                continue
            else:
                feature_vec.append(word)
    return feature_vec

In [6]:
def get_svm_vectors(train_data,feature_list):
    sorted_ftr = sorted(feature_list)
    map_ftr = {}
    ftr_vec = []
    label_vec = []
    for item in train_data:
        map_ftr={}
        
        #initialize the map to each word in feature list as 0
        for word in sorted_ftr:
            map_ftr[word]=0
        
        tweet_data = item[0]
        label_data = item[1]
        
        #Now we create a map for words in tweets to words in feature list
        for word in tweet_data:
            if word in map_ftr:
                map_ftr[word]=1
        
        #this gives a mapping of words in tweet out of all the words in feature list
        ftr_vec.append(map_ftr.values())
        
        #This adds the coorosponding label of the tweet (i.e. in favor of Clinton or Trump)
        label_vec.append(label_data)
        
    return {'feature_vector': ftr_vec,'labels':label_vec}        

In [7]:
def getSVMFeatureMap(test_data,feature_list):
    sorted_ftr = sorted(feature_list)
    map_ftr = {}
    ftr_vec = []
    for item in test_data:
        map_ftr={}
        
        #initialize the map to each word in feature list as 0
        for word in sorted_ftr:
            map_ftr[word]=0
        
        #Now we create a map for words in tweets to words in feature list
        for word in item:
            if word in map_ftr:
                map_ftr[word]=1
        
        #this gives a mapping of words in tweet out of all the words in feature list
        ftr_vec.append(map_ftr.values())
        
    return ftr_vec 

In [8]:
def process(percentage,f1name,f2name):
    dumpFile = 'election_svm.model'
    global feature_list
    train_data = []
    test_data = []
    test_labels = []
    data1 = pd.read_csv(f1name)
    size = (len(data1.index)-1)*percentage
    size/=100
    #size = 10
    label=0
    for i in range(0,size):
        feature_vec = get_features(data1.iloc[i,2])
        train_data.append((feature_vec,label))
        feature_list.extend(feature_vec)
        
    label=1
    data2 = pd.read_csv(f2name)
    for i in range(0,size):
        feature_vec = get_features(data2.iloc[i,2])
        train_data.append((feature_vec,label))
        feature_list.extend(feature_vec)
        
    #print feature_list
    feature_list = list(set(feature_list))
    #print len(feature_list)
    #print test_data
    
    #Now we train the SVM classifier
    svm_data = get_svm_vectors(train_data,feature_list)
    task = svm_problem(svm_data['labels'],svm_data['feature_vector'])
    param = svm_parameter('-q')   #used to supress the console o/p
    #param.kernel_type = SIGMOID
    classifier = svm_train(task,param)
    svm_save_model(dumpFile,classifier)
    
    #Testing
    label=0
    for i in range(size+1,len(data1)):
        test_data.append(get_features(data1.iloc[i,2]))
        test_labels.append(label)
        
    for i in range(size+1,len(data2)):
        test_data.append(get_features(data2.iloc[i,2]))
        test_labels.append(label)
    
    test_ftr_vec = getSVMFeatureMap(test_data,feature_list)
    p_labels,p_accs,p_vals = svm_predict(test_labels,test_ftr_vec,classifier)
    
    print p_labels
    print ("-------------------------------------------------------------------------------")
    
    '''training_set = nltk.classify.util.apply_features(map_features, train_data)
    
    classifier = nltk.NaiveBayesClassifier.train(training_set)
    
    missclassified_tweets=0
    for i in range(size+1,len(data1)):
        if(classifier.classify(map_features(get_features(data1.iloc[i,2])))!=0):
            missclassified_tweets+=1
    
    for i in range(size+1,len(data2)):
        if(classifier.classify(map_features(get_features(data2.iloc[i,2])))!=1):
            missclassified_tweets+=1
    print missclassified_tweets
    print len(data1)+len(data2)'''
    

In [ ]:
if __name__ == '__main__':
    per_train = 30
    f1name = "Clinton_data.csv"
    f2name = "Trump_data.csv"
    test_svm()
    while per_train <= 70:
        print 'per_train is %s' % per_train
        process(per_train,f1name,f2name)
        per_train+=10
    
    

Accuracy = 80% (4/5) (classification)
[0.0, 1.0, 1.0, 1.0, 2.0]
-------------------------------------------------------------------------------
per_train is 30
Accuracy = 73.1869% (15157/20710) (classification)
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0